# 이상형 월드컵을 통한 서울시 자취방 추천

## 1. 서울시 부동산 매물 크롤링

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv

In [20]:
rent = input("월세/전세 중 선택해주세요: ")
district = input("구를 입력해주세요: ")
neighborhood = input("동을 입력해주세요: ")

월세/전세 중 선택해주세요: 월세
구를 입력해주세요: 서대문구
동을 입력해주세요: 연희동


In [25]:
# 브라우저 생성
# browser = webdriver.Chrome('C:/chromedriver.exe')
browser = webdriver.Chrome()

# 웹사이트 열기
browser.get("https://land.naver.com/")
# 원룸.투룸 선택 하기
browser.find_element_by_link_text('원룸·투룸').click()
browser.find_element_by_link_text(rent).click()
browser.find_element_by_link_text('리스트로 보기').click()
browser.find_element_by_link_text('서울시').click()
browser.implicitly_wait(1) # 클릭 대기
browser.find_element_by_link_text(district).click()
browser.implicitly_wait(1) # 클릭 대기
browser.find_element_by_partial_link_text(neighborhood+" ").click()
browser.find_element_by_link_text("확인 매물 보기").click()
# 매물 가져오기 

# 스크롤 전 높이
itemlist = browser.find_element_by_xpath('//*[@id="listContents1"]/div')
before_h=browser.execute_script("return arguments[0].scrollHeight;",itemlist)

# 무한 스크롤(scroll in scroll)
while True:
    #맨 아래로 스크롤을 내린다.
    browser.execute_script("arguments[0].scrollBy(0,arguments[0].scrollHeight)", itemlist)
    
    # 스크롤 사이 페이지 로딩 시간
    time.sleep(1)
    
    # 스크롤 후 높이
    after_h=browser.execute_script("return arguments[0].scrollHeight;",itemlist)
    
    if after_h == before_h:
        break
    before_h = after_h
    
# 파일 생성
f = open(r"data_"+district+"_"+neighborhood+".csv",'w',encoding='utf-8-sig',newline='')

csvWriter = csv.writer(f)
csvWriter.writerow(["방이름","보증금","월세","전세","방 수","욕실 수","주소","평 수","관리비","주차 여부","방향","층","총 층","건축물 용도","전세여부","월세여부","부동산 이름","부동산 전화번호","복층 여부","사용승인일","내부시설","기타시설"])


# 매물 정보 div
items = browser.find_elements_by_css_selector('div.item')

for item in items:
    try:
        item.find_element_by_css_selector("a.label").click()
        browser.implicitly_wait(1)
        
        if item == items[0]:
            item.find_element_by_xpath('//*[@id="tabArea"]/button').click()
            
        dict = {}
        try:
            name = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[2]/h4').text
        except:
            name = item.find_element_by_css_selector("span.text").text
            
        try:
            price = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[3]/span[2]').text
            
        except:
            price = "none"
            
        if(price=="none"):
            try:
                price = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/span[2]').text
            except:
                price = "none"
                
        if(price == 'none'):
            continue
            
        find_slash = price.find('/')
        if (rent == "월세"):
            bo = price[:find_slash]
            monthly_price = price[find_slash+1:]
            yearly_price = 0
        else:
            bo = 0
            monthly_price = 0
            yearly_price = price
            
        info_list = item.find_elements_by_xpath('//*[@id="detailContents1"]/div[1]/table/tbody/tr')
        
        for i in info_list:
            key_list = i.find_elements_by_class_name('table_th')
            value_list = i.find_elements_by_class_name('table_td')
            for num in range(len(key_list)):
                dict[key_list[num].text] = value_list[num].text
        # 방수/욕실수 
        try:
            room = dict['방수/욕실수']
            find_slash = room.find('/')
            room_num = room[find_slash-1:find_slash]
            bath_num = room[find_slash+1:find_slash+2]
        except:
            room = "none"
            room_num = 0
            bath_num = 0
            
        # 주소    
        try:
            address = dict['소재지']
        except:
            address = "none"
        
        if(address == "none"):
            continue
        
        # 면적    
        try:
            size = dict['공급/전용면적']
        except:
            size = "none"
            
        if(size=='none'):
            continue
            
        # 관리비
        try:
            administrative_expenses = dict['월관리비']
        except:
            administrative_expenses = "none"
        
        # 주차공간
        try:
            parking = dict['주차가능여부']
        except:
            parking = "none"
        
        # 방향
        try:
            direction = dict['방향']
        except:
            direction = "none"
        
        # 층수
        try: 
            floor = dict['해당층/총층']
            find_slash = floor.find('/')
            floor_num = floor[:find_slash]
            all_num = floor[find_slash+1:find_slash+2]
        except:
            floor = "none"
            floor_num = "0"
            all_num = "0"
            
        # 건축물 용도    
        try:
            for_what = dict['건축물 용도']
        except:
            for_what = "none"
        
        # 부동산 정보
        try:
            real_estate = dict['중개사']
            if(real_estate.find("톡톡")!= -1):
                real_estate_name = real_estate[:real_estate.find("톡톡")]
                real_estate_name.strip('\n')
            elif(real_estate.find("길찾기")!=-1):
                real_estate_name = real_estate[:real_estate.find("길찾기")]
                real_estate_name.strip('\n')
            else:
                real_estate_name = real_estate[:real_estate.find("대표")]
                real_estate_name.strip('\n')
            
            if(real_estate.find("최근") != -1):
                real_estate_num = real_estate[real_estate.find("전화")+2:real_estate.find("최근")-1]
            elif(real_estate.find("매매")!= -1):
                real_estate_num = real_estate[real_estate.find("전화")+2:real_estate.find("매매")-1]
            else:
                real_estate_num = real_estate[real_estate.find("전화")+2:]
        except:
            real_estate_name = "none"
            real_estate_number = "none"
        
        # 복층 여부
        try:
            is_single = dict['복층여부']
        except:
            is_single = 'none'
        
        # 사용 승인일
        try:
            date = dict['사용승인일']
        except:
            date = 'none'
            
        # 월세, 전세 
        if (rent == "월세"):
            monthly = 1
            yearly = 0
        else:
            monthly = 0
            yearly = 1
            
        print("방 이름: "+ name)
        print("방 수: " + room_num)
        print("욕실 수: "+ bath_num)
        print("월세: "+ str(monthly_price))
        print("보증금: "+str(bo))
        print("전세: "+str(yearly_price))
        print("주소: "+ address)
        print("평수: "+ size)
        print("관리비: "+ administrative_expenses)
        print("주차: " + parking)
        print("방향: "+ direction)
        print("해당층: "+floor_num)
        print("총 층: "+all_num)
        print("건축물 용도: "+for_what)
        print("월세: " +monthly)
        print("전세: " +yearly)
        print("부동산 정보: "+real_estate_name)
        print("부동산 전화번호: "+ real_estate_num)
        print("복층여부: "+is_single)
        print("사용승인일: "+date)
        
        # 내부시설
        try:
            option = item.find_element_by_xpath('//*[@id="detailContents1"]/div[1]/div/div[2]')
            option_list = option.find_elements_by_css_selector('div.option_item')
            option_data = ""
            print("내부시설: ",end="")
            for option in option_list:
                option_data += option.text+", "
                print(option.text+", ",end="")
            print(" ")
        except:
            option_data = "none"
            print("내부시설: none")
                
        # 기타시설
        try:
            etc = item.find_element_by_xpath('//*[@id="detailContents1"]/div[1]/div/div[4]')
            etc_list = etc.find_elements_by_css_selector('div.option_item')
            etc_data = ""
            print("보안 및 기타시설: ",end="")
            for etc in etc_list:
                etc_data += etc.text + ", "
                print(etc.text+", ",end="")
            print(" ")
        except:
            etc_data = "none"
            print("보안 및 기타시설: none")
        
        csvWriter.writerow([name,bo,monthly_price,yearly_price,room_num,bath_num,address,size,administrative_expenses,parking,direction,floor_num,all_num,for_what,yearly,monthly,real_estate_name,real_estate_num,is_single,date,option_data,etc_data])

        
    except:
        item.find_element_by_css_selector("a.item_link").click()
        browser.implicitly_wait(1)
        
        if item == items[0]:
            item.find_element_by_xpath('//*[@id="tabArea"]/button').click()
        dict = {}
        
        try:
            name = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[2]/h4').text
        except:
            name = item.find_element_by_css_selector("span.text").text
            
        try:
            price = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[3]/span[2]').text
        except:
            price = "none"
            
        if(price=="none"):
            try:
                price = item.find_element_by_xpath('//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/span[2]').text
            except:
                price = "none"
        
        if(price == 'none'):
            continue
            
        find_slash = price.find('/')
        if (rent == "월세"):
            bo = price[:find_slash]
            monthly_price = price[find_slash+1:]
            yearly_price = 0
        else:
            bo = 0
            monthly_price = 0
            yearly_price = price
            
        info_list = item.find_elements_by_xpath('//*[@id="detailContents1"]/div[1]/table/tbody/tr')
        
        for i in info_list:
            key_list = i.find_elements_by_class_name('table_th')
            value_list = i.find_elements_by_class_name('table_td')
            for num in range(len(key_list)):
                dict[key_list[num].text] = value_list[num].text
        # 방수/욕실수 
        try:
            room = dict['방수/욕실수']
            find_slash = room.find('/')
            room_num = room[find_slash-1:find_slash]
            bath_num = room[find_slash+1:find_slash+2]
        except:
            room = "none"
            room_num = 0
            bath_num = 0
            
        # 주소    
        try:
            address = dict['소재지']
        except:
            address = "none"
        
        if(address == "none"):
            continue
        
        # 면적    
        try:
            size = dict['공급/전용면적']
        except:
            size = "none"
            
        if (size=="none"):
            continue
            
        # 관리비
        try:
            administrative_expenses = dict['월관리비']
        except:
            administrative_expenses = "none"
        
        # 주차공간
        try:
            parking = dict['주차가능여부']
        except:
            parking = "none"
        
        # 방향
        try:
            direction = dict['방향']
        except:
            direction = "none"
        
        # 층수
        try: 
            floor = dict['해당층/총층']
            find_slash = floor.find('/')
            floor_num = floor[:find_slash]
            all_num = floor[find_slash+1:find_slash+2]
        except:
            floor = "none"
            floor_num = "0"
            all_num = "0"
            
        # 건축물 용도    
        try:
            for_what = dict['건축물 용도']
        except:
            for_what = "none"
        
        # 부동산 정보
        try:
            real_estate = dict['중개사']
            if(real_estate.find("톡톡")!= -1):
                real_estate_name = real_estate[:real_estate.find("톡톡")]
                real_estate_name.strip('\n')
            elif(real_estate.find("길찾기")!=-1):
                real_estate_name = real_estate[:real_estate.find("길찾기")]
                real_estate_name.strip('\n')
            else:
                real_estate_name = real_estate[:real_estate.find("대표")]
                real_estate_name.strip('\n')
             
            if(real_estate.find("최근") != -1):
                real_estate_num = real_estate[real_estate.find("전화")+2:real_estate.find("최근")-1]
            elif(real_estate.find("매매")!= -1):
                real_estate_num = real_estate[real_estate.find("전화")+2:real_estate.find("매매")-1]
            else:
                real_estate_num = real_estate[real_estate.find("전화")+2:]
        except:
            real_estate_name = "none"
            real_estate_number = "none"
        
        # 복층 여부
        try:
            is_single = dict['복층여부']
        except:
            is_single = 'none'
            
        # 사용 승인일
        try:
            date = dict['사용승인일']
        except:
            date = 'none'
            
        # 월세, 전세 
        if (rent == "월세"):
            monthly = 1
            yearly = 0
        else:
            monthly = 0
            yearly = 1
            
        print("방 이름: "+ name)
        print("방 수: " + room_num)
        print("욕실 수: "+ bath_num)
        print("월세: "+ str(monthly_price))
        print("보증금: "+str(bo))
        print("전세: "+str(yearly_price))
        print("주소: "+ address)
        print("평수: "+ size)
        print("관리비: "+ administrative_expenses)
        print("주차: " + parking)
        print("방향: "+ direction)
        print("해당층: "+floor_num)
        print("총 층: "+all_num)
        print("건축물 용도: "+for_what)
        print("월세/전세 "+rent)
        print("부동산 정보: "+real_estate_name)
        print("부동산 전화번호: "+ real_estate_num)
        print("복층여부: "+is_single)
        print("사용승인일: "+date)
    
        # 내부시설
        try:
            option = item.find_element_by_xpath('//*[@id="detailContents1"]/div[1]/div/div[2]')
            option_list = option.find_elements_by_css_selector('div.option_item')
            option_data = ""
            print("내부시설: ",end="")
            for option in option_list:
                option_data += option.text+", "
                print(option.text+", ",end="")
            print(" ")
        except:
            option_data = "none"
            print("내부시설: none")

        # 기타시설
        try:
            etc = item.find_element_by_xpath('//*[@id="detailContents1"]/div[1]/div/div[4]')
            etc_list = etc.find_elements_by_css_selector('div.option_item')
            etc_data = ""
            print("보안 및 기타시설: ",end="")
            for etc in etc_list:
                etc_data += etc.text + ", "
                print(etc.text+", ",end="")
            print(" ")
        except:
            etc_data = "none"
            print("보안 및 기타시설: none")
        
        csvWriter.writerow([name,bo,monthly_price,yearly_price,room_num,bath_num,address,size,administrative_expenses,parking,direction,floor_num,all_num,for_what,yearly,monthly,real_estate_name,real_estate_num,is_single,date,option_data,etc_data])

    print(" ")
# 파일 닫기
f.close()


C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20356/1343830492.py:8: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text('원룸·투룸').click()
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20356/1343830492.py:9: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text(rent).click()
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20356/1343830492.py:10: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text('리스트로 보기').click()
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20356/1343830492.py:11: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_e

WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x0099B8F3+2406643]
	Ordinal0 [0x0092AF31+1945393]
	Ordinal0 [0x0081C610+837136]
	Ordinal0 [0x00810442+787522]
	Ordinal0 [0x00810C78+789624]
	Ordinal0 [0x008124B2+795826]
	Ordinal0 [0x0080BF09+769801]
	Ordinal0 [0x0081DAC0+842432]
	Ordinal0 [0x00873E62+1195618]
	Ordinal0 [0x00864096+1130646]
	Ordinal0 [0x0083E636+976438]
	Ordinal0 [0x0083F546+980294]
	GetHandleVerifier [0x00C09612+2498066]
	GetHandleVerifier [0x00BFC920+2445600]
	GetHandleVerifier [0x00A34F2A+579370]
	GetHandleVerifier [0x00A33D36+574774]
	Ordinal0 [0x00931C0B+1973259]
	Ordinal0 [0x00936688+1992328]
	Ordinal0 [0x00936775+1992565]
	Ordinal0 [0x0093F8D1+2029777]
	BaseThreadInitThunk [0x7639FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77B77A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77B77A4E+238]


## 2. 데이터 전처리

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [5]:
df = pd.read_csv("data_강북구_우이동.csv",encoding = '')
df.head()

,방이름,보증금,월세,전세,방 수,욕실 수,주소,평 수,관리비,주차 여부,...,총 층,건축물 용도,전세여부,월세여부,부동산 이름,부동산 전화번호,복층 여부,사용승인일,내부시설,기타시설
0,일반원룸3층,500,40,0,1,1,서울시 강북구 우이동 71-21,6.05평/5.74평(전용률95%),5만원,불가능,...,4,제2종 근린생활시설,0,1,수유역공인중개사사무소,"02-996-7900,010-5250-8779",단층,2005.02.23,"냉장고, 세탁기, 신발장, 싱크대, 인덕션레인지, 벽걸이에어컨,","인터폰,"
1,일반원룸3층,500,40,0,1,1,서울시 강북구 우이동 72-66,5.29평/3.49평(전용률66%),5만원,불가능,...,3,단독주택,0,1,수유역공인중개사사무소,"02-996-7900,010-5250-8779",단층,2016.10.06,"냉장고, 세탁기, 신발장, 싱크대, 옷장, 인덕션레인지, 책상, 침대, 벽걸이에어컨,","인터폰, 현관보안, 무인택배함,"
2,다가구2층,"4,000",60,0,2,1,서울시 강북구 우이동 21-8,12.97평/12.97평(전용률100%),0원,불가능,...,3,미등기건물,0,1,번다공인중개사,"02-995-4300,010-4061-9123",단층,-,none,none
3,다가구2층,"4,000",60,0,2,1,서울시 강북구 우이동 124-33,15.02평/15.02평(전용률100%),0원,불가능,...,3,단독주택,0,1,대우공인중개사사무소,"02-997-1881,02-997-1881",단층,1996.09.19,"베란다,",none
4,일반원룸고층,300,29,0,1,1,서울시 강북구 우이동 61-19,6.35평/6.35평(전용률100%),5만원,불가능,...,4,단독주택,0,1,토마토부동산공인중개사사무소,"02-905-1356,010-7518-1356",단층,2011.11,"침대, 책상, 옷장, 냉장고, 세탁기, 샤워부스, 싱크대, 가스레인지, 인덕션레인지...","현관보안,"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   방이름       35 non-null     object
 1   보증금       35 non-null     object
 2   월세        35 non-null     int64 
 3   전세        35 non-null     int64 
 4   방 수       35 non-null     int64 
 5   욕실 수      35 non-null     object
 6   주소        35 non-null     object
 7   평 수       35 non-null     object
 8   관리비       35 non-null     object
 9   주차 여부     35 non-null     object
 10  방향        35 non-null     object
 11  층         35 non-null     object
 12  총 층       35 non-null     int64 
 13  건축물 용도    35 non-null     object
 14  전세여부      35 non-null     int64 
 15  월세여부      35 non-null     int64 
 16  부동산 이름    35 non-null     object
 17  부동산 전화번호  35 non-null     object
 18  복층 여부     35 non-null     object
 19  사용승인일     35 non-null     object
 20  내부시설      35 non-null     object
 21  기타시설      35 non-n

### 방수

In [7]:
def convert_str_to_int(x):
    convert = int(x)
    return convert

In [8]:
df['방 수'] = df['방 수'].apply(convert_str_to_int)
type("방 수")

str

### 복층여부

In [9]:
df['복층 여부']

0       단층
1       단층
2       단층
3       단층
4       단층
5       단층
6       단층
7       단층
8       단층
9     none
10      단층
11    none
12      단층
13      단층
14      단층
15      단층
16      단층
17      단층
18      단층
19      단층
20      단층
21      단층
22      단층
23    none
24      단층
25    none
26      단층
27      단층
28      단층
29      단층
30      단층
31      단층
32      단층
33      단층
34      단층
Name: 복층 여부, dtype: object

In [10]:
def convert_floor(x):
    if (x=='단층'or x=='none'):
        return 0
    elif (x=='복층'):
        return 1

In [11]:
df['복층 여부'] = df['복층 여부'].apply(convert_floor)
df['복층 여부']

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
Name: 복층 여부, dtype: int64

###  평수

In [12]:
df['평 수']

0        6.05평/5.74평(전용률95%)
1        5.29평/3.49평(전용률66%)
2     12.97평/12.97평(전용률100%)
3     15.02평/15.02평(전용률100%)
4       6.35평/6.35평(전용률100%)
5       23.01평/3.81평(전용률17%)
6        5.29평/3.49평(전용률66%)
7        6.05평/5.14평(전용률85%)
8       6.99평/6.99평(전용률100%)
9       4.87평/4.87평(전용률100%)
10       1.81평/1.51평(전용률83%)
11     14.66평/13.11평(전용률89%)
12      23.52평/5.88평(전용률25%)
13      23.52평/5.88평(전용률25%)
14       7.26평/4.95평(전용률68%)
15       6.95평/6.35평(전용률91%)
16      23.01평/3.81평(전용률17%)
17      23.01평/3.81평(전용률17%)
18      23.52평/4.53평(전용률19%)
19       5.74평/5.14평(전용률89%)
20       5.29평/5.14평(전용률97%)
21    10.99평/10.99평(전용률100%)
22    15.02평/15.02평(전용률100%)
23     18.31평/14.55평(전용률79%)
24       5.44평/4.53평(전용률83%)
25      4.87평/4.87평(전용률100%)
26      1.88평/1.88평(전용률100%)
27      5.13평/5.13평(전용률100%)
28            5평/5평(전용률100%)
29      23.06평/7.86평(전용률34%)
30       9.07평/6.65평(전용률73%)
31      9.98평/9.98평(전용률100%)
32      6.95평/6.95평(전용률100%)
33      9.35평/9.35평(전용률100%)
34      3.63평/

In [13]:
def convert_size(x):
    find_slash = x.find('/')
    first_pyung = x.find('평')
    second_pyung = x.find('평',first_pyung+1)
    size = x[find_slash+1:second_pyung]
    return size

In [19]:
df['평 수'] = df['평 수'].apply(convert_size)

### 욕실수

In [20]:
for i in range(len(df)):
    if df.loc[i, '욕실 수'] == '-':
        df.loc[i, '욕실 수'] = 1
        
df['욕실 수'] = df['욕실 수'].astype('int64')

In [21]:
df['욕실 수']

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    1
32    1
33    1
34    1
Name: 욕실 수, dtype: int64

### 사용승인일

In [28]:
for i in range(len(df)):
    if df.loc[i, '사용승인일'] == "none" or df.loc[i,'사용승인일']=="-":
        df.loc[i, '사용승인일'] = "0"
        
for i in range(len(df)):
    df.loc[i, '사용승인일'] = df.loc[i, '사용승인일'][:4]


df['사용승인일'] = df['사용승인일'].astype('int64')

In [29]:
date_sum = 0
for i in range(len(df)):
    if df.loc[i, '사용승인일'] != 0:
        date_sum += df.loc[i, '사용승인일']

for i in range(len(df)):
    if df.loc[i, '사용승인일'] == 0:
        df.loc[i, '사용승인일'] = math.trunc(date_sum / len(df))

In [30]:
df['사용승인일']

0     2005
1     2016
2     1720
3     1996
4     2011
5     2012
6     2018
7     2009
8     2019
9     2013
10    2016
11    1996
12    2009
13    2009
14    2012
15    2019
16    2012
17    2012
18    2009
19    2016
20    2016
21    1990
22    1996
23    1995
24    2017
25    2013
26    1997
27    1720
28    1720
29    1720
30    1720
31    1983
32    1978
33    1994
34    2017
Name: 사용승인일, dtype: int64

### 관리비

In [25]:
for i in range(len(df)):
    if df.loc[i, '관리비'] == "none":
        df.loc[i, '관리비'] = "0"
    df.loc[i, '관리비'] = df.loc[i, '관리비'].replace("만원", "")
    df.loc[i, '관리비'] = df.loc[i, '관리비'].replace("원", "")

df['관리비'] = df['관리비'].astype('float64')

In [26]:
cost_sum = 0
for i in range(len(df)):
    if df.loc[i, '관리비'] != 0:
        cost_sum += df.loc[i, '관리비']

for i in range(len(df)):
    if df.loc[i, '관리비'] == 0:
        df.loc[i, '관리비'] = math.trunc(cost_sum / len(df))

In [27]:
df['관리비']

0     5.0
1     5.0
2     2.0
3     2.0
4     5.0
5     2.0
6     5.0
7     4.0
8     6.0
9     3.0
10    2.0
11    2.0
12    5.0
13    5.0
14    5.0
15    6.0
16    2.0
17    2.0
18    2.0
19    2.0
20    2.0
21    2.0
22    2.0
23    2.0
24    5.0
25    3.0
26    2.0
27    5.0
28    2.0
29    3.0
30    5.0
31    2.0
32    2.0
33    2.0
34    2.0
Name: 관리비, dtype: float64